In [6]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 72.4 MB/s eta 0:00:00


In [2]:
import spacy

# Load spaCy's English NLP model
nlp = spacy.load("en_core_web_sm")

In [7]:
import fitz
import re
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [66]:

cleaned_text = ""  # Initialize an empty string to accumulate all the cleaned text
footer_text = "©2009 Project Management Institute. Practice Standard for Project Risk Management"
current_chapter = None  # To track the current chapter

# Open the PDF document
pdf_document = fitz.open('practice-standard-project-risk-management.pdf')

# Loop through the pages starting from page 13
for page_number in range(13, pdf_document.page_count):
    page = pdf_document[page_number]
    text_dict = page.get_text("dict")

    if text_dict:
        # Remove footer and lines containing 'Figure'
        cleaned_page_text = []

        for block in text_dict["blocks"]:
            # Check if the block is a text block
            if "lines" in block:  # It's a text block
                for line in block["lines"]:
                    # Get the bounding box of the line to check its position
                    bbox = line["bbox"]

                    # Check if the line is not in the bottom-right area (e.g., last 50 units of height)
                    if bbox[3] < page.rect.height - 50:  # Avoid last 50 units height as footer
                        line_text = " ".join([span["text"] for span in line["spans"]])
                        # Exclude the footer text
                        if footer_text not in line_text and 'Figure' not in line_text:
                            cleaned_page_text.append(line_text)

        # Join all cleaned lines into a single string for this page
        cleaned_page_text = "\n".join(cleaned_page_text)

        # Split the cleaned text into lines for further processing
        lines = cleaned_page_text.split('\n')

        for line in lines:
            # Strip whitespace for comparison
            stripped_line = line.strip()

            # Skip lines that are only numbers
            if stripped_line.isdigit():
                continue  # Skip this line

            # Use regex to match chapter titles while ignoring any trailing page numbers
            chapter_match = re.match(r'^(CHAPTER\s+\d+)', stripped_line)

            # Check if the line contains a chapter title
            if chapter_match:
                # Extract the chapter title without page numbers
                chapter_title = chapter_match.group(1)

                # Only add the chapter title if it's different from the current one
                if current_chapter is None or chapter_title != current_chapter:
                    current_chapter = chapter_title  # Update the current chapter
                    cleaned_text += chapter_title + "\n"  # Include the chapter title in the cleaned text
            else:
                # Include other lines only if they are not empty
                if stripped_line:  # Avoid adding empty lines
                    cleaned_text += stripped_line + "\n"  # Add non-title lines

# Close the PDF document
pdf_document.close()

# Output the cleaned text
print(cleaned_text)

CHAPTER 1
1.1 Purpose of the  Practice Standard for Project Risk Management
The purpose of the   Practice Standard for Project Risk Management   is to (  a   ) provide a standard for project
management practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that
are recognized as good practice on most projects most of the time and (  b   ) provide a standard that is globally
applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for
training or educational purposes.
The   Practice Standard for Project Risk Management   covers risk management as it is applied to single
projects only. Like the   PMBOK    ®    Guide   – Fourth Edition, this practice standard does not cover risk in programs
or portfolios of projects.
Chapter 11 of the   PMBOK    ®    Guide –   Fourth Edition, is the basis for the   Practice Standard for Project Risk
Management  . This practice standard is consistent with that chapter, emph

In [67]:
def divide_into_sections(text):
    sections = text.split('CHAPTER')
    organized_paragraphs = []  # Initialize a list to hold paragraphs of each chapter
    full_text = ""  # Initialize a variable to hold the full text summary

    for section in sections[1:]:  # Skip the first split part as it will be before the first chapter
        lines = section.split('\n')

        if len(lines) == 0:
            continue  # Skip empty sections

        current_section = None  # Initialize to None
        paragraphs = []

        for line in lines[1:]:  # Start from the second line to skip the chapter title
            line = line.strip()  # Strip whitespace from each line

            # Check if the line is not empty, has more than 1 character, is not just symbols,
            # and does not contain only the bullet point symbol
            if line and len(line) > 1 and not all(char in ' .-_•' for char in line):
                paragraphs.append(line)  # Add non-empty lines to paragraphs
                full_text += line + "\n"  # Add line to full text summary

                # Check for section headers (e.g., "1.1", "1.2")
                if line[0].isdigit() and '.' in line:
                    # If we have a current section, save it before moving on to the new one
                    if current_section is not None:  # Ensure current_section is valid
                        organized_paragraphs.append(paragraphs)  # Add the list of paragraphs to organized_paragraphs
                        paragraphs = []  # Reset paragraphs for the next section
                    current_section = line  # Update current_section to the new section header

        # Add the last section if it exists
        if paragraphs:  # If there are remaining paragraphs
            organized_paragraphs.append(paragraphs)  # Add the remaining paragraphs

    return organized_paragraphs, full_text.strip()  # Return both organized paragraphs and full text summary

In [68]:
organized_text, full_text = divide_into_sections(cleaned_text)
print("Full Text Summary:")
print(full_text)

Full Text Summary:
1.1 Purpose of the  Practice Standard for Project Risk Management
The purpose of the   Practice Standard for Project Risk Management   is to (  a   ) provide a standard for project
management practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that
are recognized as good practice on most projects most of the time and (  b   ) provide a standard that is globally
applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for
training or educational purposes.
The   Practice Standard for Project Risk Management   covers risk management as it is applied to single
projects only. Like the   PMBOK    ®    Guide   – Fourth Edition, this practice standard does not cover risk in programs
or portfolios of projects.
Chapter 11 of the   PMBOK    ®    Guide –   Fourth Edition, is the basis for the   Practice Standard for Project Risk
Management  . This practice standard is consistent with that chap

In [11]:
print(organized_text)

{'1': {'1.1 Purpose of the  Practice Standard for Project Risk Management': ['The purpose of the   Practice Standard for Project Risk Management   is to (  a   ) provide a standard for project', 'management practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that', 'are recognized as good practice on most projects most of the time and (  b   ) provide a standard that is globally', 'applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for', 'training or educational purposes.', 'The   Practice Standard for Project Risk Management   covers risk management as it is applied to single', 'projects only. Like the   PMBOK    ®    Guide   – Fourth Edition, this practice standard does not cover risk in programs', 'or portfolios of projects.', 'Chapter 11 of the   PMBOK    ®    Guide –   Fourth Edition, is the basis for the   Practice Standard for Project Risk', 'Management  . This practice standard is consi

In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
doc = nlp(full_text)

print(doc)

1.1 Purpose of the  Practice Standard for Project Risk Management
The purpose of the   Practice Standard for Project Risk Management   is to (  a   ) provide a standard for project
management practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that
are recognized as good practice on most projects most of the time and (  b   ) provide a standard that is globally
applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for
training or educational purposes.
The   Practice Standard for Project Risk Management   covers risk management as it is applied to single
projects only. Like the   PMBOK    ®    Guide   – Fourth Edition, this practice standard does not cover risk in programs
or portfolios of projects.
Chapter 11 of the   PMBOK    ®    Guide –   Fourth Edition, is the basis for the   Practice Standard for Project Risk
Management  . This practice standard is consistent with that chapter, emphasizing th

In [15]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\[.*?\]', '', text)  # Remove references
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with digits
    text = re.sub('http\S+\s', ' ', text)

    # Tokenize, lemmatize, and remove stop words using spaCy
    doc = nlp(text)

    # Return lemmatized tokens that are not stop words or punctuations
    tokens = [token.lemma_.strip() for token in doc if not token.is_stop and not token.is_punct]

    return tokens

In [16]:
def extract_most_common_ngrams(text, n=3, top_n=3):
    """Extract the most common n-grams from the text, keeping the longer one when counts are the same."""
    processed_tokens = preprocess_text(text)

    ngrams = []

    # Extract n-grams from the preprocessed tokens
    for i in range(len(processed_tokens)):
        for j in range(2, n + 1):  # From bi-grams (2) to n-grams (n)
            if i + j <= len(processed_tokens):
                ngram = " ".join(processed_tokens[i:i+j]).strip()
                ngrams.append(ngram)

    # Count the occurrences of each n-gram
    ngram_counts = Counter(ngrams)

    # Create a dictionary to keep the longest n-gram for each count
    unique_ngrams = {}
    for ngram, count in ngram_counts.items():
        if count >= 10:  # Only consider n-grams that occur 10 times or more
            if count not in unique_ngrams:
                unique_ngrams[count] = ngram  # If count not in unique_ngrams, add n-gram
            else:
                # If the same count exists, keep the longer n-gram
                if len(ngram) > len(unique_ngrams[count]):
                    unique_ngrams[count] = ngram

    # Create a list of (ngram, count) tuples, filtering out those with count < 10
    filtered_ngrams = [(ngram, count) for count, ngram in unique_ngrams.items() if count >= 10]

    # Sort by count in descending order and get the most common n-grams
    most_common = sorted(filtered_ngrams, key=lambda x: x[0], reverse=True)[:top_n]

    return most_common, processed_tokens

In [17]:
from collections import Counter
most_common_ngrams, tokens = extract_most_common_ngrams(full_text, n=4, top_n=100)

# Display the most common n-grams
print("Most Common N-grams:")
for phrase, count in most_common_ngrams:
    print(f" ******'{phrase}' occurs {count} times \n")


Most Common N-grams:
 ******'weakness' occurs 69 times 

 ******'understanding' occurs 32 times 

 ******'trigger condition' occurs 49 times 

 ******'tool technique' occurs 56 times 

 ******'template example present' occurs 18 times 

 ******'technique example template' occurs 12 times 

 ******'technique  strength' occurs 57 times 

 ******'technique' occurs 133 times 

 ******'strength  weakness' occurs 64 times 

 ******'standard project risk management' occurs 14 times 

 ******'stakeholder' occurs 54 times 

 ******'risk response' occurs 136 times 

 ******'risk monitoring control' occurs 33 times 

 ******'risk management process' occurs 81 times 

 ******'risk management planning' occurs 21 times 

 ******'risk management plan' occurs 61 times 

 ******'risk management activity' occurs 23 times 

 ******'risk management' occurs 420 times 

 ******'risk identi cation' occurs 78 times 

 ******'risk identi' occurs 80 times 

 ******'risk analysis' occurs 166 times 

 ******'risk

In [89]:
def custom_tokenizer(nlp, most_common_phrases):
    """Create a custom tokenizer that treats phrases as single tokens."""
    # Create a copy of spaCy's tokenizer
    prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)
    infix_re = spacy.util.compile_infix_regex(nlp.Defaults.infixes)

    # Tokenizer function that replaces phrases with underscores
    def custom_tokenizer_function(text):
        # Replace phrases in the text with underscore versions
        for phrase, _ in most_common_phrases:
            # Make sure to match the exact phrase boundaries
            text = re.sub(r'\b' + re.escape(phrase) + r'\b', phrase.replace(" ", "_"), text)
        # Use spaCy's default tokenizer to handle the rest
        return Tokenizer(nlp.vocab, prefix_search=prefix_re.search, suffix_search=suffix_re.search,
                         infix_finditer=infix_re.finditer)(text)

    return custom_tokenizer_function

In [90]:
from spacy.tokenizer import Tokenizer

nlp.tokenizer = custom_tokenizer(nlp, most_common_ngrams)

In [84]:
organized_text[0]

['1.1 Purpose of the  Practice Standard for Project Risk Management',
 'The purpose of the   Practice Standard for Project Risk Management   is to (  a   ) provide a standard for project',
 'management practitioners and other stakeholders that deﬁ nes the aspects of Project Risk Management that',
 'are recognized as good practice on most projects most of the time and (  b   ) provide a standard that is globally',
 'applicable and consistently applied. This practice standard has a descriptive purpose rather than one used for',
 'training or educational purposes.',
 'The   Practice Standard for Project Risk Management   covers risk management as it is applied to single',
 'projects only. Like the   PMBOK    ®    Guide   – Fourth Edition, this practice standard does not cover risk in programs',
 'or portfolios of projects.',
 'Chapter 11 of the   PMBOK    ®    Guide –   Fourth Edition, is the basis for the   Practice Standard for Project Risk',
 'Management  . This practice standard is co

In [97]:
cleaned_chapters = []  # To store cleaned text for each chapter
cleaned_chapter_tokens = []  # To store tokens for each chapter
chapter_docs=[]


# Iterate over each chapter in organized_text
for chapter in organized_text:
    # Join the paragraphs of the chapter into a single string
    result_str = " ".join(chapter)  # Joining all paragraphs in the chapter

    # Preprocess the full text of the chapter
    cleaned_tokens = preprocess_text(result_str)
    cleaned_text = ' '.join(cleaned_tokens)  # Join cleaned tokens back into a string

    # Process the cleaned text with spaCy
    doc = nlp(cleaned_text)
    chapter_docs.append(doc)

    # Extract tokens, removing stop words and punctuation
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text.strip()]

    # Append cleaned text and tokens to their respective lists
    cleaned_chapters.append(cleaned_text)
    cleaned_chapter_tokens.append(tokens)

# Now you have cleaned_chapters and cleaned_chapter_tokens for each chapter

In [98]:
cleaned_chapter_tokens[0]

['purpose',
 'practice_standard',
 'project',
 'risk_management',
 'purpose',
 'practice_standard',
 'project',
 'risk_management',
 'provide',
 'standard',
 'project_management',
 'practitioner',
 'stakeholder',
 'de',
 'ne',
 'aspect',
 'project',
 'risk_management',
 'recognize',
 'good',
 'practice',
 'project',
 'time',
 'b',
 'provide',
 'standard',
 'globally',
 'applicable',
 'consistently',
 'apply',
 'practice_standard',
 'descriptive',
 'purpose',
 'training',
 'educational',
 'purpose',
 'practice_standard',
 'project',
 'risk_management',
 'cover',
 'risk_management',
 'apply',
 'single',
 'project',
 'like',
 'pmbok__guide',
 'fourth',
 'edition',
 'practice_standard',
 'cover',
 'risk',
 'program',
 'portfolio',
 'project',
 'chapter',
 'pmbok__guide',
 'fourth',
 'edition',
 'basis',
 'practice_standard',
 'project',
 'risk_management',
 'practice_standard',
 'consistent',
 'chapter',
 'emphasize',
 'concept',
 'principle',
 'relate',
 'project',
 'risk_management',
 'a

In [93]:
cleaned_tokens = preprocess_text(full_text)
cleaned_text =  ' '.join(cleaned_tokens)

doc = nlp(cleaned_text)
tokens = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text.strip()]

In [94]:
print(tokens)

['purpose', 'practice_standard', 'project', 'risk_management', 'purpose', 'practice_standard', 'project', 'risk_management', 'provide', 'standard', 'project', 'management', 'practitioner', 'stakeholder', 'de', 'ne', 'aspect', 'project', 'risk_management', 'recognize', 'good', 'practice', 'project', 'time', 'b', 'provide', 'standard', 'globally', 'applicable', 'consistently', 'apply', 'practice_standard', 'descriptive', 'purpose', 'training', 'educational', 'purpose', 'practice_standard', 'project', 'risk_management', 'cover', 'risk_management', 'apply', 'single', 'project', 'like', 'pmbok__guide', 'fourth', 'edition', 'practice_standard', 'cover', 'risk', 'program', 'portfolio', 'project', 'chapter', 'pmbok__guide', 'fourth', 'edition', 'basis', 'practice_standard', 'project_risk', 'management', 'practice_standard', 'consistent', 'chapter', 'emphasize', 'concept', 'principle', 'relate', 'project', 'risk_management', 'align', 'pmi', 'practice_standard', 'textbook', 'handbook', 'course',

In [99]:
token_details_all_chapters = []  # List to hold token details for all chapters

for doc in chapter_docs:
    unique_tokens = set()  # Set to track unique tokens for the current chapter
    token_details = []  # List to hold token details for the current chapter

    for idx, token in enumerate(doc):
        if not token.is_space and not token.is_stop and not token.is_punct and token.text.strip():
            # Only add token details if the token is not a duplicate
            if token.text not in unique_tokens:
                token_details.append([idx, token.text, token.lemma_, token.pos_, token.tag_, token.dep_])
                unique_tokens.add(token.text)  # Track the token as seen

    token_details_all_chapters.append(token_details)  # Append the details for the current chapter

# Now, token_details_all_chapters contains the token details for each chapter

In [100]:
token_details_all_chapters[0]

[[1, 'purpose', 'purpose', 'NOUN', 'NN', 'compound'],
 [3, 'practice_standard', 'practice_standard', 'NOUN', 'NN', 'compound'],
 [4, 'project', 'project', 'NOUN', 'NN', 'compound'],
 [5, 'risk_management', 'risk_management', 'NOUN', 'NN', 'compound'],
 [12, 'provide', 'provide', 'VERB', 'VBP', 'ROOT'],
 [13, 'standard', 'standard', 'ADJ', 'JJ', 'amod'],
 [14, 'project_management', 'project_management', 'ADJ', 'JJ', 'amod'],
 [15, 'practitioner', 'practitioner', 'NOUN', 'NN', 'compound'],
 [16, 'stakeholder', 'stakeholder', 'NOUN', 'NN', 'nmod'],
 [17, 'de', 'de', 'PROPN', 'NNP', 'compound'],
 [18, 'ne', 'ne', 'PROPN', 'NNP', 'compound'],
 [19, 'aspect', 'aspect', 'PROPN', 'NNP', 'compound'],
 [22, 'recognize', 'recognize', 'VERB', 'VBP', 'conj'],
 [23, 'good', 'good', 'ADJ', 'JJ', 'amod'],
 [24, 'practice', 'practice', 'NOUN', 'NN', 'compound'],
 [26, 'time', 'time', 'NOUN', 'NN', 'compound'],
 [28, 'b', 'b', 'NOUN', 'NN', 'nsubj'],
 [32, 'globally', 'globally', 'ADV', 'RB', 'advmod'],

In [45]:
token_details = []
unique_tokens = set()

for idx, token in enumerate(doc):
    if not token.is_space and not token.is_stop and not token.is_punct and token.text.strip():
        # Only add token details if the token is not a duplicate
        if token.text not in unique_tokens:
            token_details.append([idx, token.text, token.lemma_, token.pos_, token.tag_, token.dep_])
            unique_tokens.add(token.text)  # Add the token to the set to track it as seen

In [101]:
token_details_all_chapters = []  # List to hold token details for all chapters

for doc in chapter_docs:
    token_details = []  # List to hold token details for the current chapter

    for idx, token in enumerate(doc):
        if not token.is_space and not token.is_stop and not token.is_punct and token.text.strip():
            token_details.append([idx, token.text, token.lemma_, token.pos_, token.tag_, token.dep_])

    token_details_all_chapters.append(token_details)

In [105]:
from tabulate import tabulate

# Assuming token_details_all_chapters contains token details for all chapters
for chapter_index, token_details in enumerate(token_details_all_chapters):
    print(f"Chapter {chapter_index + 1}:")
    print(tabulate(token_details[:25], headers=['ID', 'TEXT', 'LEMMA', 'POS', 'TAG', 'DEP']))
    print("\n")  # Add a newline for better separation between chapters


Chapter 1:
  ID  TEXT                LEMMA               POS    TAG    DEP
----  ------------------  ------------------  -----  -----  --------
   1  purpose             purpose             NOUN   NN     compound
   3  practice_standard   practice_standard   NOUN   NN     compound
   4  project             project             NOUN   NN     compound
   5  risk_management     risk_management     NOUN   NN     compound
   6  purpose             purpose             NOUN   NN     compound
   8  practice_standard   practice_standard   NOUN   NN     compound
   9  project             project             NOUN   NN     compound
  10  risk_management     risk_management     NOUN   NN     nsubj
  12  provide             provide             VERB   VBP    ROOT
  13  standard            standard            ADJ    JJ     amod
  14  project_management  project_management  ADJ    JJ     amod
  15  practitioner        practitioner        NOUN   NN     compound
  16  stakeholder         stakeholder      

In [106]:
from spacy import displacy
sentences = list(chapter_docs[0].sents)[:10]
for sent in sentences:
  displacy.render(sent,style='dep',jupyter=True,options={'distance':100})

In [60]:
doc_cleaned = nlp(cleaned_text_rejoined)

In [127]:
for token in chapter_docs[6]:
    if token.pos_ == "VERB":
        verb_phrase = " ".join([token.text] + [child.text for child in token.children])
        print(f"Verb Phrase: {verb_phrase}")

Verb Phrase: nongovernment government environment
Verb Phrase: find example check compare
Verb Phrase: check describe speci
Verb Phrase: c
Verb Phrase: compare organization project nition


In [123]:
# Extract POS tags including stop words
pos_tags = [(token.text, token.pos_) for token in chapter_docs[0] if not token.is_punct and not token.is_space]

# Display the first 20 POS tags
print(pos_tags[:20])


[('purpose', 'NOUN'), ('practice_standard', 'NOUN'), ('project', 'NOUN'), ('risk_management', 'NOUN'), ('purpose', 'NOUN'), ('practice_standard', 'NOUN'), ('project', 'NOUN'), ('risk_management', 'NOUN'), ('provide', 'VERB'), ('standard', 'ADJ'), ('project_management', 'ADJ'), ('practitioner', 'NOUN'), ('stakeholder', 'NOUN'), ('de', 'PROPN'), ('ne', 'PROPN'), ('aspect', 'PROPN'), ('project', 'PROPN'), ('risk_management', 'PROPN'), ('recognize', 'VERB'), ('good', 'ADJ')]


In [130]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# Use TF-IDF to find the most important terms
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 4))  # Include n-grams in TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform([full_text])  # Fit on the full text
feature_names = tfidf_vectorizer.get_feature_names_out()
dense = tfidf_matrix.todense()
denselist = dense.tolist()
tfidf_results = list(zip(feature_names, denselist[0]))

# Sort by TF-IDF score
tfidf_results = sorted(tfidf_results, key=lambda x: x[1], reverse=True)

# Combine TF-IDF results with n-grams while avoiding duplicates
combined_results = []
seen_terms = set()  # Set to track added terms

# Add TF-IDF results to the combined list
for term, score in tfidf_results:
    if term not in seen_terms:  # Check for duplicates
        combined_results.append((term, score))
        seen_terms.add(term)  # Add term to seen set

# Add n-grams and their scores
for phrase, count in most_common_ngrams:
    if phrase not in seen_terms:  # Check for duplicates
        # Look up the TF-IDF score for the n-gram, if it exists
        ngram_score = next((score for term, score in tfidf_results if term == phrase), 0)  # Default to 0 if not found
        combined_results.append((phrase, ngram_score))
        seen_terms.add(phrase)  # Add phrase to seen set

# Sort the combined results by score
combined_results = sorted(combined_results, key=lambda x: x[1], reverse=True)

# Display top 50 results
print("Combined TF-IDF and Most Common N-grams (No Duplicates):")
for term, score in combined_results[:50]:
    print(f"{term}: {score:.4f}")  # Print score with 4 decimal places

print("Processing complete.")

Combined TF-IDF and Most Common N-grams (No Duplicates):
risk: 0.6130
project: 0.4809
management: 0.2643
risks: 0.2081
risk management: 0.1850
project risk: 0.1305
analysis: 0.1204
process: 0.1052
pmp: 0.1046
project risk management: 0.0956
plan: 0.0872
identiﬁ: 0.0754
objectives: 0.0692
responses: 0.0692
risk analysis: 0.0652
response: 0.0534
project management: 0.0489
techniques: 0.0489
stakeholders: 0.0484
ed: 0.0461
quantitative: 0.0456
deﬁ: 0.0450
impact: 0.0450
cation: 0.0444
information: 0.0444
overall: 0.0439
actions: 0.0427
cost: 0.0427
identify: 0.0427
identiﬁ cation: 0.0427
level: 0.0416
speciﬁ: 0.0405
used: 0.0399
probability: 0.0377
planning: 0.0371
time: 0.0371
tools: 0.0371
plan risk: 0.0360
management plan: 0.0354
standard: 0.0354
processes: 0.0349
effective: 0.0332
organization: 0.0326
project objectives: 0.0326
practice: 0.0321
quantitative risk: 0.0321
risk responses: 0.0321
effect: 0.0315
individual: 0.0315
quantitative risk analysis: 0.0315
Processing complete.
